## Library imports

In [1]:
import numpy as np
import pandas as pd
from chromatography import ExperimentAnalytes
from separation_utility import *
from torch import optim, tensor
import matplotlib.pyplot as plt
%matplotlib qt

## Dataset Import

In [2]:
sample = 'GilarSample'
alist = pd.read_csv(f'../data/{sample}.csv')
# GilarSample
# Peterpeptides
# Roca
# Peter32
# Eosin
# Alizarin
# Controlmix1
# Controlmix2
#alist['k0'] = np.exp(alist.lnk0)

## Useful functions definition

In [3]:
def loss_field(exp, taus, N = 200):
    phis = np.linspace(0, 1, N)
    losses = np.zeros((N, N))
    j = 0
    for phi1 in phis:
        i = 0
        for phi2 in phis:
            exp.reset()
            exp.run_all([phi1, phi2], taus)
            losses[i, j] = exp.loss()
            i += 1
        j += 1
    X, Y = np.meshgrid(phis, phis)
    
    return X, Y, losses

In [8]:
# Parameters
run_time_lim = 10.0
sigma_max = 0.2
delta_taus = [.25, .25, 10]
num_episodes = 5000
batch_size = 10
lr = .1
optim = lambda a, b: torch.optim.SGD(a, b, momentum=0.65)
lr_decay_factor = .5
lr_milestones = 1000
print_every= 100
baseline = 0.55
max_norm = 2.
beta = .0
weights = [1., 1.]
delta_loss = True

In [13]:
exp = ExperimentAnalytes(k0 = alist.k0.values, S = alist.S.values, h=0.001, run_time=10.0)
pol = PolicySingle(len(delta_taus), sigma_max = sigma_max)
losses, best_program, mus, sigmas = reinforce_one_set(
        exp, 
        pol, 
        delta_taus= delta_taus, 
        num_episodes=num_episodes, 
        batch_size=batch_size,
        optim=optim,
        lr=lr, 
        print_every=print_every,
        lr_decay_factor=lr_decay_factor,
        lr_milestones=lr_milestones,
        baseline=baseline,
        max_norm=max_norm,
        beta=beta,
        #lim=lim
    )

Loss: 0.21917262400235593, epoch: 100/5000
Loss: 0.1933540187897394, epoch: 200/5000
Loss: 0.21800885394255456, epoch: 300/5000
Loss: 0.2103708075015016, epoch: 400/5000
Loss: 0.21859972022571802, epoch: 500/5000
Loss: 0.41286449703036593, epoch: 600/5000
Loss: 0.40024844039780766, epoch: 700/5000
Loss: 0.4225629241950073, epoch: 800/5000
Loss: 0.35540817409455977, epoch: 900/5000
Loss: 0.31439197574269817, epoch: 1000/5000
Loss: 0.26112815417235463, epoch: 1100/5000
Loss: 0.1784829513703677, epoch: 1200/5000
Loss: 0.20756976277400535, epoch: 1300/5000
Loss: 0.17514398795474478, epoch: 1400/5000
Loss: 0.19543088448388474, epoch: 1500/5000
Loss: 0.2039204068533916, epoch: 1600/5000
Loss: 0.18828188188041456, epoch: 1700/5000
Loss: 0.22337924966415335, epoch: 1800/5000
Loss: 0.25539112528987484, epoch: 1900/5000
Loss: 0.2565234634176091, epoch: 2000/5000
Loss: 0.2350288897047228, epoch: 2100/5000
Loss: 0.19216414546388955, epoch: 2200/5000
Loss: 0.17579152039584384, epoch: 2300/5000
Loss

In [15]:
exp.reset()
exp.run_all(pol.mu.detach().numpy(), delta_taus)
exp.print_analytes(rc=(7, 5), angle=45,  title=f'Solvent Strength function\nLoss: {round(exp.loss(),4)}')
plt.show()
plt.legend()
#plt.savefig(f'results/{sample}_result_{run}.png')

In [ ]:
exp.reset()
exp.run_all(best_program.numpy(), taus)
exp.print_analytes(rc=(7, 5), angle=45,  title=f'Solvent Strength function\nLoss: {round(exp.loss(),4)}')
plt.legend()
plt.savefig(f'results/{sample}_best_result_{run}.png')

In [ ]:
plt.plot(mus[:, 0], label='Mu: phi1')
plt.plot(mus[:, 1], label='Mu: phi2')
#plt.plot(mus[:, 2], label='Mu: phi3')
#plt.plot(mus[:, 3], label='Mu: phi4')
plt.ylim((0,1))
plt.legend()
plt.savefig(f'results/{sample}_mus_{run}.png')

In [ ]:
plt.plot(sigmas[:, 0], label='Sigma: phi1')
plt.plot(sigmas[:, 1], label='Sigma: phi2')
#plt.plot(sigmas[:, 2], label='Sigma: phi3')
#plt.plot(sigmas[:, 3], label='Sigma: phi4')
plt.ylim((0,sigma_max))
plt.legend()
plt.savefig(f'results/{sample}_sigmas_{run}.png')

In [ ]:
plt.plot(np.arange(print_every, num_episodes + print_every, print_every), losses)
plt.xlabel("Episodes")
plt.ylabel("Loss")
plt.title("Loss during learning process")
plt.savefig(f'results/{sample}_loss_{run}.png')

In [ ]:
X, Y, Loss_field = loss_field(exp, [.25, 30000], N = 500)
plt.contourf(X, Y, Loss_field, levels=100)
plt.xlabel("Phi1")
plt.ylabel("Phi2")
index = np.unravel_index(np.argmin(Loss_field), Loss_field.shape)
plt.scatter(X[index], Y[index], c="r", s=5)
plt.title(f"Loss Field, change of phi at 0.5\nGlobal min ({round(np.min(Loss_field), 3)}) at {round(X[index], 4), round(Y[index], 4)}, red dot")
#plt.savefig(f'results/{sample}_loss_filed_at_5.png')

In [ ]:
N = -1
mlab.surf(X.T[:N, :N], Y.T[:N, :N], Loss_field.T[:N, :N])

In [7]:

phis = np.linspace(0, 1, 1000)
losses = []
for phi1 in phis:
    exp.reset()
    exp.run_all([phi1], [10])
    losses.append(exp.loss())
plt.plot(losses[:])
plt.ylim((0, 2))

(0, 2)

In [ ]:
plt.contourf(X, Y, Loss_field, levels=100)

In [ ]:
exp_time = ExperimentAnalytes(k0 = alist.k0.values, S = alist.S.values, h=0.001, grad='iso', run_time=10.0)
pol_time = PolicyTime(2, sigma_max = .2)
losses_time, best_program_time, mus_time, sigmas_time, n_par_time = reinforce_delta_tau(
        exp_time, 
        pol_time,
        num_episodes=5000, 
        batch_size=10,
        optim=lambda a, b: torch.optim.SGD(a, b, momentum=0.65),
        lr=.1, 
        print_every=100,
        lr_decay=lambda a, b, c: step_decay(a, b, c, steps=5, decay_factor=0.5),
        baseline=0.65,
        max_norm=2.
    )

In [ ]:
exp_time.reset()
exp_time.run_all(pol_time.mu.detach().numpy()[:2], [0.25, 3000000])
exp_time.print_analytes(rc=(7, 5))
exp_time.loss()

In [ ]:
exp_time.reset()
exp_time.run_all(best_program_time[0], best_program_time[1])
exp_time.print_analytes(rc=(7, 5))
exp_time.loss()

In [ ]:
plt.plot(losses_time)

In [ ]:
plt.plot(mus_time[:, 0], label='Mu: 0')
plt.plot(mus_time[:, 1], label='Mu: 1')
plt.plot(mus_time[:, 2], label='Mu: 2')
plt.legend()
plt.show()

In [ ]:
plt.plot(sigmas_time[:, 0], label='Sigma: 0')
plt.plot(sigmas_time[:, 1], label='Sigma: 1')
plt.plot(sigmas_time[:, 2], label='Sigma: 2')
plt.plot(sigmas_time[:, 3], label='Sigma: 3')
plt.plot(sigmas_time[:, 4], label='Sigma: 4')
plt.plot(sigmas_time[:, 5], label='Sigma: 5')
plt.plot(sigmas_time[:, 6], label='Sigma: 6')

plt.legend()
plt.show()